Scraping the race times of:http://www.olympicgamesmarathon.com/results.php

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

First obtain the HTML file of the main site. This site contains all the links to the actual datasets for each year Olympic Games where held.

In [2]:
url = "http://www.olympicgamesmarathon.com/results.php"
site = requests.get(url)

Now BeautifulSoup will be used to parse the HTML structure.

In [3]:
soup = BeautifulSoup(site.content, "html.parser")

In [4]:
# link.attrs can be used to obtain a dictionary with the key and value of alll the attributes a tag has
for link in soup.find_all("a")[:20]:
    print(link.attrs['href'].find("olympiad"))

-1
-1
-1
-1
-1
-1
-1
-1
0
0
0
0
0
0
0
0
0
0
0
0


We see from this that we only want these links whereby the href contains olympiad in it. However, some links occur multiple times. So probably best to first select all the olympiad names. And then make a unique list from this.

In [5]:
links = []
# obtain all olympiad.php's
# find method returns -1 in case the substring is not found
for link in soup.find_all("a"):
    if link.attrs["href"].find("olympiad") >= 0:
        links.append(link.attrs["href"])

# make the list unique
links = set(links)

# add 'http://www.olympicgamesmarathon.com/' in front of each element of the list 
full_links = []
prefix = 'http://www.olympicgamesmarathon.com/'
for link in links:
    full_links.append(prefix + link)

The next step is no to obtain the HTML structure of each of these links. Then to parse these html structures to finally obtain the data stored in each of these pages. This has to happen all inside a for loop. I thinks it will be best to write the code for one year. Once this code is fullfilled then this code will be generic for all the years.

In [6]:
link = full_links[0]
page = requests.get(link)
soup = BeautifulSoup(page.content, "html.parser")

There is also a specific table containing information about the people that did not finish. As I don't know the target value for these athletes it is useless to store these atheletes. Therefore, i will only create a table containing information about the athelest whereof i have their race time.

In [51]:
for link in full_links:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")


    # Assign dfName to variable
    dfName = 'Runtimes_' +  re.search("\d+", link).group() + '.csv'

    # find the indices where the table containing the racers starst and ends
    start = 0
    end = 0
    for index, value in enumerate(soup.find_all("td")):
        if value.text == "1°":
            start = index
        if value.text == "Did not finish":
            end = index - 1
            break

    # Creating an empty Dataframe with column names only
    dfObj = pd.DataFrame(columns=['Position', 'FirstName', "LastName","Nationality", "Time"])

    # each innerlist is one specific racer with it results 
    racers = []
    l = 0
    for i in range(5, end + 4, 5):
        racers.append(soup.find_all("td")[start:end][l:i])
        l += 5

    # converting attributes to text
    for racer in racers:
        if len(racer) == 0:
            break
        else:
            for i in range(5):
                if i == 0:
                    racer[i] = racer[i].text[:-1]
                else:
                    racer[i] = racer[i].text

    # filling in the dataframe
    for racer in racers:
        if len(racer) == 0:
            break
        else:
            dfObj = dfObj.append({'Position':racer[0] , 'FirstName':racer[1], 'LastName':racer[2], 'Nationality':racer[3],
                              'Time':racer[4]}, ignore_index=True)
    
    # write the pandas dataframe to csv file with correct name 
    dfObj.to_csv(r'C:\Users\BrechtDewilde\Documents\UGENT -  statistical data analysis\STATISTICAL DATA ANALYSIS\thesis\R analysis\Data\scraped\race times\ ' + dfName, index = False)